In [2]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

In [3]:
df = pd.read_csv("Data_Final1.csv")
df = df[df['Year_Month'] <= '2020-12']

In [4]:
df

,Year,Month,Year_Month,Crime type,LAD19CD,LAD19NM,CTYNM,RGN19CD,RGN19NM,CNTY21NM,NTN21NM,Frequency,Population,Population Density,Rainfall
0,2013,5,2013-05,Anti-social behaviour,E06000001,Hartlepool,Cleveland,E12000001,North East,England,UK,622,92662.0,987.224740,76.6
1,2013,5,2013-05,Anti-social behaviour,E06000002,Middlesbrough,Cleveland,E12000001,North East,England,UK,1063,138911.0,2578.408789,76.6
2,2013,5,2013-05,Anti-social behaviour,E06000003,Redcar and Cleveland,Cleveland,E12000001,North East,England,UK,842,134960.0,551.093101,76.6
3,2013,5,2013-05,Anti-social behaviour,E06000004,Stockton-on-Tees,Cleveland,E12000001,North East,England,UK,1146,193433.0,948.523664,76.6
4,2013,5,2013-05,Anti-social behaviour,E06000005,Darlington,Durham,E12000001,North East,England,UK,474,105726.0,535.389019,76.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397996,2020,12,2020-12,Violence and sexual offences,E09000029,Sutton,Metropolitan,E12000007,London,England,UK,314,207707.0,4736.740083,123.6
397997,2020,12,2020-12,Violence and sexual offences,E09000030,Tower Hamlets,Metropolitan,E12000007,London,England,UK,755,331969.0,16792.402250,123.6
397998,2020,12,2020-12,Violence and sexual offences,E09000031,Waltham Forest,Metropolitan,E12000007,London,England,UK,574,276940.0,7135.054362,123.6
397999,2020,12,2020-12,Violence and sexual offences,E09000032,Wandsworth,Metropolitan,E12000007,London,England,UK,590,329735.0,9623.674587,123.6


In [5]:
df = df.rename({
'Crime type' : 'Crime_Type',
'LAD19CD': 'LAD_Codes', 
'LAD19NM': 'LAD_Names',
'CTYNM': 'County',
'CNTY21NM': 'Country',
'NTN21NM':'Nation',
'Population Density':'Population_Density'}, axis=1)

In [6]:
year_slider = pn.widgets.IntSlider(name='Year slider', start=2013, end=2020, step=1, value=2014)
year_slider

IntSlider(end=2020, name='Year slider', start=2013, value=2014)

In [7]:
select_rgn = pn.widgets.Select(name='Select', options=['North East', 'North West', 'Yorkshire and The Humber',
       'East Midlands', 'West Midlands', 'South West', 'East of England',
       'South East', 'London'])

In [8]:
idf = df.interactive()

In [9]:
crimeyr_pipeline = (
    idf[
        (idf.Year == year_slider) &
        (idf.RGN19NM == select_rgn)
    ].groupby(['Year', 'Crime_Type'])['Frequency'].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='Crime_Type') 
    .reset_index(drop=True)
)

In [10]:
type(crimeyr_pipeline)

hvplot.interactive.Interactive

In [11]:
cryr_plot = crimeyr_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 14, layout='fit_columns',show_index=False) 
cryr_plot

In [12]:
select_typ = pn.widgets.Select(name='Select', options=['Anti-social behaviour', 'Bicycle theft', 'Burglary',
       'Criminal damage and arson', 'Drugs', 'Other crime', 'Other theft',
       'Possession of weapons', 'Public order', 'Robbery', 'Shoplifting',
       'Theft from the person', 'Vehicle crime',
       'Violence and sexual offences'])

In [13]:
crimeyrtp_pipeline = (
    idf[
        (idf.Year == year_slider) &
        (idf.Crime_Type == select_typ)
    ].groupby(['Year', 'RGN19NM'])['Frequency'].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='RGN19NM') 
    .reset_index(drop=True)
)

In [14]:
cryrtyp_plot = crimeyrtp_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 9, layout='fit_columns',show_index=False) 
cryrtyp_plot

In [15]:
crimergn_pipeline = (
    idf[(idf.RGN19NM == select_rgn )].groupby(['Year_Month'])['Frequency'].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='Year_Month') 
    .reset_index(drop=True)
)

In [16]:
crrgn_plot = crimergn_pipeline.hvplot(x = 'Year_Month', y='Frequency',line_width=2, title="Total Crime Frequency plot for Region across 2013-2020")
crrgn_plot

In [17]:
crimetyp_pipeline = (
    idf[(idf.Crime_Type == select_typ )].groupby(['Year_Month'])['Frequency'].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='Year_Month') 
    .reset_index(drop=True)
)

In [18]:
crtyp_plot = crimetyp_pipeline.hvplot(x = 'Year_Month', y='Frequency',line_width=2, title="Total Crime Frequency plot for crime type across 2013-2020")
crtyp_plot

In [19]:
template = pn.template.FastListTemplate(
    title='Crime frequency:England dashboard', 
    sidebar=[pn.pane.Markdown("# Crime frequency England dashboard"), 
             pn.pane.Markdown("#### Choose from the options below"), 
             pn.pane.Markdown("## Settings"),   
             year_slider,select_rgn,select_typ],
    main=[pn.Row(pn.Column(cryr_plot.panel(width=260), margin=(0,25)), 
                 pn.Column(crrgn_plot.panel(width=700))), 
          pn.Row(pn.Column(cryrtyp_plot.panel(width=260), margin=(0,25)),
              pn.Column(crtyp_plot.panel(width=700))
                 )],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
template.show()
#template.servable();

Launching server at http://localhost:61803
